In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from collections import Counter
from imblearn.over_sampling import SMOTE
from datetime import datetime
from sklearn.decomposition import PCA

In [43]:
# from google.colab import drive
# drive.mount('/content/drive')

In [44]:
# os.chdir('/content/drive/MyDrive/AI SGS Competition')

In [45]:
path = "Data/train"
os.listdir(path)
df1 = pd.DataFrame()
for file in os.listdir(path):
    if (file.endswith('.csv')) & (file.startswith('19')):
        df = pd.read_csv(path + "/"+ file)

        df.drop(['Filament'], axis=1, inplace=True) # Drop Filament column
        df.drop(['Emission'], axis=1, inplace=True) # Drop Emission column
        df.drop(['Temp'], axis=1, inplace=True) # Drop Temp column
        df.drop(['Test ID'], axis=1, inplace=True) # Drop Test ID column

        df['502_to_69_ratio'] = df['502 m/z'] / df['69 m/z'] # Create new column with ratio of 502 m/z to 69 m/z

        ############################################
        #clean the datetime format
        def clean_date(date):
            if 'UTC' in date:
                date = date.split('(')[0]
                date = date.split("  ")[0]
            return date

        def convert_to_datetime(date):
            date_string = date
            date_format = '%d/%m/%Y %I:%M %p'

            datetime_obj = datetime.strptime(date_string, date_format)
            formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
            return formatted_datetime

        df['Tune Date'] = df['Tune Date'].apply(lambda x: clean_date(x))
        if 'UTC' in df['Tune Date'][0]:
            df['Tune Date'] = df['Tune Date'].apply(lambda x: convert_to_datetime(x))

        df['Tune Date'] = pd.to_datetime(df['Tune Date'])
        df['total_hour_diff'] = df['Tune Date'].diff().apply(lambda x: x.total_seconds()/3600) # new column with total hour difference

        df.drop(['Tune Date'], axis=1, inplace=True) # Drop Tune Date column
        ############################################

        # Perform PCA on the highly correlated columns
        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['69 m/z', '70 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 69 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['219 m/z', '220 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 219 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['502 m/z', '503 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 502 m/z'] = components

        # add columns for pct change
        def precent_change(col_name , periods):
            df[col_name + " pct change " + str(periods) + 'days'] = df[col_name].pct_change(periods=periods) * 100
            return df[col_name + " pct change " + str(periods) + 'days']

        for i in range (10):
            precent_change('EM Volts', i+1)
        for i in range (10):
            precent_change('502_to_69_ratio', i+1)

        df1 = pd.concat([df1,df])

df1.fillna(0, inplace=True) # fill all NaN values with 0

# # Perform one-hot encoding
# one_hot_encoded = pd.get_dummies(df1['System'], prefix='System')
# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# df1 = pd.concat([df1, one_hot_encoded], axis=1)
#drop System column
df1.drop(['System'], axis=1, inplace=True)
df1.drop(['ID'], axis=1, inplace=True)
df1

,69 m/z,70 m/z,219 m/z,220 m/z,502 m/z,503 m/z,Amu gain,Amu offset,EM Volts,Ent Lens,...,502_to_69_ratio pct change 1days,502_to_69_ratio pct change 2days,502_to_69_ratio pct change 3days,502_to_69_ratio pct change 4days,502_to_69_ratio pct change 5days,502_to_69_ratio pct change 6days,502_to_69_ratio pct change 7days,502_to_69_ratio pct change 8days,502_to_69_ratio pct change 9days,502_to_69_ratio pct change 10days
0,389888,4815,352320,15620,24096,2770,2503,127,1718,7.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,322880,3532,359616,15261,27592,2709,2496,126,1812,12.0,...,38.272862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,316608,3490,379840,16608,29992,3163,2491,126,1718,12.0,...,10.851483,53.277517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,296128,3154,368064,15829,28832,2954,2491,125,1765,12.0,...,2.780743,13.933977,57.539771,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,300544,2958,375296,16720,29136,2995,2490,125,1812,12.0,...,-0.430444,2.338329,13.443555,56.861651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,440512,5046,449856,19000,59912,5996,2478,127,1706,12.0,...,35.227228,14.444050,15.234680,13.936579,22.482853,-13.188058,-23.393296,-26.061671,-18.294253,-26.566907
15,449920,5018,409472,18504,53848,5196,2480,127,1659,14.5,...,-12.000905,18.998737,0.709728,1.405475,0.263158,7.783801,-23.606277,-32.586794,-34.934940,-28.099682
16,381056,4220,242816,9745,28904,2786,2485,128,1659,14.5,...,-36.622535,-44.228404,-24.581617,-36.172727,-35.731780,-36.455752,-31.689359,-51.583595,-57.275219,-58.763414
17,358720,3942,226816,10157,31592,3328,2480,128,1706,12.0,...,16.105391,-26.415346,-35.246171,-12.435191,-25.893095,-25.381132,-26.221702,-20.687663,-43.785943,-50.394226


In [46]:
path = "Data/test"
val = pd.DataFrame()
for file in os.listdir(path):
    if (file.endswith('.csv')) & (file.startswith('19')):
        df = pd.DataFrame()
        df = pd.read_csv(path + "/"+ file)

        # df.drop(['Filament'], axis=1, inplace=True) # Drop Filament column
        df.drop(['Emission'], axis=1, inplace=True) # Drop Emission column
        df.drop(['Temp'], axis=1, inplace=True) # Drop Temp column
        df.drop(['Test ID'], axis=1, inplace=True) # Drop Test ID column

        df['502_to_69_ratio'] = df['502 m/z'] / df['69 m/z'] # Create new column with ratio of 502 m/z to 69 m/z

        ############################################
        #clean the datetime format
        def clean_date(date):
            if 'UTC' in date:
                date = date.split('(')[0]
                date = date.split("  ")[0]
            return date

        def convert_to_datetime(date):
            date_string = date
            date_format = '%d/%m/%Y %I:%M %p'

            datetime_obj = datetime.strptime(date_string, date_format)
            formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
            return formatted_datetime

        df['Tune Date'] = df['Tune Date'].apply(lambda x: clean_date(x))
        if 'UTC' in df['Tune Date'][0]:
            df['Tune Date'] = df['Tune Date'].apply(lambda x: convert_to_datetime(x))

        df['Tune Date'] = pd.to_datetime(df['Tune Date'])
        df['total_hour_diff'] = df['Tune Date'].diff().apply(lambda x: x.total_seconds()/3600) # new column with total hour difference

        df.drop(['Tune Date'], axis=1, inplace=True) # Drop Tune Date column
        ############################################

        # Perform PCA on the highly correlated columns
        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['69 m/z', '70 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 69 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['219 m/z', '220 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 219 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['502 m/z', '503 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 502 m/z'] = components


        # add columns for pct change
        def precent_change(col_name , periods):
            df[col_name + " pct change " + str(periods) + 'days'] = df[col_name].pct_change(periods=periods) * 100
            return df[col_name + " pct change " + str(periods) + 'days']

        for i in range (10):
            precent_change('EM Volts', i+1)
        for i in range (10):
            precent_change('502_to_69_ratio', i+1)

        val = pd.concat([val,df])
val.fillna(0, inplace=True) # fill all NaN values with 0
val

# Perform one-hot encoding
# one_hot_encoded = pd.get_dummies(val['System'], prefix='System')
# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# val = pd.concat([val, one_hot_encoded], axis=1)
#drop System column
val.drop(['System'], axis=1, inplace=True)
val.drop(['ID'], axis=1, inplace=True)
val

[0.99995022]
[0.99982566]
[0.99910726]


,69 m/z,70 m/z,219 m/z,220 m/z,502 m/z,503 m/z,Amu gain,Amu offset,EM Volts,Ent Lens,...,502_to_69_ratio pct change 1days,502_to_69_ratio pct change 2days,502_to_69_ratio pct change 3days,502_to_69_ratio pct change 4days,502_to_69_ratio pct change 5days,502_to_69_ratio pct change 6days,502_to_69_ratio pct change 7days,502_to_69_ratio pct change 8days,502_to_69_ratio pct change 9days,502_to_69_ratio pct change 10days
0,419520,5269,487104,21144,50248,4975,2489,129,1612,14.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,369088,4382,444928,20096,53792,5274,2489,126,1612,14.5,...,21.680688,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,307008,3461,399808,17592,45376,4613,2484,125,1565,16.5,...,1.411861,23.398649,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,398336,4652,462272,20504,63296,6117,2489,126,1612,9.5,...,7.510329,9.028225,32.666294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,360128,4190,455616,19944,59552,6173,2492,126,1612,12.0,...,4.066933,11.882702,13.462330,38.061743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,400576,4717,472320,21016,54976,5819,2488,125,1612,14.5,...,-17.005608,-13.630281,-7.143631,-5.832629,14.583505,0.000000,0.000000,0.000000,0.000000,0.000000
6,359296,4019,366144,16352,41752,4441,2492,126,1518,9.5,...,-15.328610,-29.727494,-26.869559,-21.377222,-20.267178,-2.980554,0.000000,0.000000,0.000000,0.000000
7,373376,4189,348480,15238,28624,3235,2488,126,1612,12.0,...,-34.028095,-44.140671,-53.639890,-51.754455,-48.131056,-47.398739,-35.994423,0.000000,0.000000,0.000000
8,396032,4511,424000,18520,49152,4918,2487,126,1659,14.5,...,61.892597,6.803630,-9.567882,-24.946413,-21.894034,-16.028019,-14.842452,3.620290,0.000000,0.000000
9,387328,5573,394112,17136,45576,4704,2485,127,1706,14.5,...,-5.191692,53.487633,1.258715,-14.262839,-28.842964,-25.949055,-20.387585,-19.263569,-1.759356,0.000000


In [47]:
X_train = df1.drop(['Condition'], axis=1)
y_train = df1['Condition']
X_val = val.drop(['Condition'], axis=1)
y_val = val['Condition']


In [48]:
X_val.shape, X_train.shape

((40, 39), (114, 39))

In [49]:
X_train.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [50]:
X_val.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [51]:
X_train.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [52]:
# apply standard scalar
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [53]:
# # apply over sampling
# from imblearn.over_sampling import SMOTE
# from collections import Counter

# sm = SMOTE(random_state=42)
# print('Original dataset shape %s' % Counter(y_train))
# X_train, y_train = sm.fit_resample(X_train, y_train)
# print('Resampled dataset shape %s' % Counter(y_train))


In [54]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential


In [55]:
X_train.shape[1]

39

In [58]:
def train_tf_model(X_train, y_train, X_val, y_val, neurons, dropout_rate, epochs, learning_rate=0.01, batch_size = 32, shapesize = X_train.shape[1]):


    model = tf.keras.Sequential([
        tf.keras.layers.Dense(neurons, activation='relu', input_shape=(shapesize,)),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(neurons, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(neurons, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam (learning_rate), loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

    history = model.fit(X_train, y_train,
                        epochs=epochs, batch_size=batch_size,
                        validation_data=( X_val, y_val ))

    return history, model

In [59]:
epochs = 11
neurons = 32
dropout_rate = 0.3
learning_rate = 0.0001
batch_size = 16
shapesize = X_train.shape[1]

history, model = train_tf_model(X_train, y_train, X_val, y_val, neurons=neurons, dropout_rate=dropout_rate, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)

y_pred = model.predict(X_val)
y_pred

Epoch 1/11
8/8 [==============================] - 2s 77ms/step - loss: 0.8170 - accuracy: 0.4035 - auc: 0.3101 - precision: 0.0312 - recall: 0.2500 - val_loss: 0.7958 - val_accuracy: 0.2500 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/11
8/8 [==============================] - 0s 7ms/step - loss: 0.7994 - accuracy: 0.4737 - auc: 0.3314 - precision: 0.0185 - recall: 0.1250 - val_loss: 0.7761 - val_accuracy: 0.3250 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/11
8/8 [==============================] - 0s 8ms/step - loss: 0.7849 - accuracy: 0.4474 - auc: 0.3408 - precision: 0.0339 - recall: 0.2500 - val_loss: 0.7571 - val_accuracy: 0.3500 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/11
8/8 [==============================] - 0s 7ms/step - loss: 0.7626 - accuracy: 0.4386 - auc: 0.2895 - precision: 0.0333 - recall: 0.2500 - val_loss: 0.7388 - val_accuracy: 0.3500 - val_auc: 0.0000e

array([[0.41618934],
       [0.47568646],
       [0.44927293],
       [0.4084194 ],
       [0.429613  ],
       [0.3489476 ],
       [0.5003372 ],
       [0.32966346],
       [0.48991776],
       [0.44704568],
       [0.38513795],
       [0.4618233 ],
       [0.45353556],
       [0.5393071 ],
       [0.34728536],
       [0.5069969 ],
       [0.4742699 ],
       [0.42530575],
       [0.5665482 ],
       [0.4481543 ],
       [0.558589  ],
       [0.48774996],
       [0.44926152],
       [0.53832775],
       [0.51302713],
       [0.5566325 ],
       [0.35983726],
       [0.37139297],
       [0.48569602],
       [0.27269313],
       [0.45883787],
       [0.5192785 ],
       [0.50288075],
       [0.49052575],
       [0.5212269 ],
       [0.5324924 ],
       [0.5003207 ],
       [0.45374587],
       [0.57969123],
       [0.5664965 ]], dtype=float32)

In [33]:
# epochs = 100
# neurons = [16,32,64]
# dropout = [0,0.3,0.5]
# learning_rate = [0.01, 0.001, 0.0001]
# batch_size = [16, 32]
# df_DNN_result = pd.DataFrame()

# result_df = pd.DataFrame(columns=['neurons', 'dropout', 'learning_rate', 'batch_size', 'accuracy', 'val_accuracy', 'auc', 'val_auc'])

# for n in neurons:
#     for d in dropout:
#         for l in learning_rate:
#             for b in batch_size:
#                 history, model = train_tf_model(X_train, y_train, X_val, y_val, neurons=n, dropout_rate=d, epochs=epochs, learning_rate=l, batch_size=b)
#                 df_DNN_result = df_DNN_result.append(pd.DataFrame(history.history).assign(neurons=n, dropout=d, learning_rate=l, batch_size=b))
#                 print(f'neurons: {n}, dropout: {d}, learning_rate: {l}, batch_size: {b}')
#                 print(f'accuracy: {history.history["accuracy"][-1]}, val_accuracy: {history.history["val_accuracy"][-1]}')
#                 print(f'auc: {history.history["auc"][-1]}, val_auc: {history.history["val_auc"][-1]}')
#                 print(f'precision: {history.history["precision"][-1]}, val_precision: {history.history["val_precision"][-1]}')
#                 print(f'recall: {history.history["recall"][-1]}, val_recall: {history.history["val_recall"][-1]}')


#                 result = pd.DataFrame(history.history)

#                 result['neurons'] = n
#                 result['dropout'] = d
#                 result['learning_rate'] = l
#                 result['batch_size'] = b
#                 result_df = pd.concat([result_df, result], axis=0)
#                 print('')

In [34]:
# def my_f1():
# 2 * (precision * recall) / (precision + recall)

In [35]:
# result_df.to_csv('Result/time10 34 system.csv')